In [6]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from sentence_transformers import SentenceTransformer
from chromadb import Client
from chromadb.utils import Documents, Vectors
from typing import List
import asyncio


C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


ModuleNotFoundError: No module named 'chromadb'

In [ ]:
app = FastAPI()
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
chroma_client = Client(persist_directory="chromadb_data")
collection = chroma_client.get_or_create_collection("document_embeddings")


In [ ]:
async def embed_text(text: str):
    embedding = embedding_model.encode(text)
    return embedding

async def extract_text(file: UploadFile):
    # Implementation for extracting text from PDF, DOC, DOCX, and TXT.
    # Assume we return plain text from the uploaded file
    return "Extracted text content"


In [3]:
@app.post("/ingest")
async def ingest_document(file: UploadFile = File(...)):
    # Extract and embed text asynchronously
    text_content = await extract_text(file)
    embedding = await embed_text(text_content)
    
    # Store document and embedding in ChromaDB
    doc_id = f"doc_{file.filename}"
    collection.add(documents=Documents([text_content]), vectors=Vectors([embedding]), ids=[doc_id])
    return JSONResponse(content={"message": "Document ingested successfully", "doc_id": doc_id})

@app.post("/query")
async def query_documents(query: str):
    # Embed the query
    query_embedding = await embed_text(query)
    
    # Perform the search
    results = collection.query(query_embedding, top_k=5)  # Adjust top_k as needed
    
    return JSONResponse(content={"results": results})


NameError: name 'app' is not defined